In [1]:
import pandas as pd
import json
import re
from pprint import pp
from tree_sitter_languages import get_parser
import os

parser = get_parser("python")

processed_count = 0


projects_path = "/home/XXX/Traces/Python/All/tried"


projects_list = os.listdir(projects_path)
print(len(projects_list))

544


In [62]:
pydata = []

for project in projects_list[500:545]:
    project_path = os.path.join(projects_path, project)
    trace_file_path = os.path.join(project_path, "collected_traces.json")
    
    if os.path.isfile(trace_file_path):
        try:
            with open(trace_file_path) as f:
                project_data = json.load(f)
            pydata.append((project, project_data))
        except json.JSONDecodeError as e:
            print(f"Skipping {project}: {e}")



Skipping dlukes+corpy: Unterminated string starting at: line 201671126 column 11 (char 4419028848)


In [63]:
data = []

for project_name, trace in pydata:
    for k, v in trace.items():
        code_files = {}
        code_funcs = {}
        fpath = v[0]
        if "/local/rcs/XXX/code/pytrace-collector/logs/pypibugs/tried/" in fpath:
            fpath = fpath.replace(
                "/local/rcs/XXX/code/pytrace-collector/logs/pypibugs/tried/", 
                "/home/XXX/Traces/Python/All/tried/"
            )
        elif "/local/rcs/XXX/code/pytrace-collector/logs/self_collected/tried/" in fpath:
            fpath = fpath.replace(
                "/local/rcs/XXX/code/pytrace-collector/logs/self_collected/tried/", 
                "/home/XXX/Traces/Python/All/tried/"
            )

        
        try:
            with open(fpath, encoding='utf-8') as f:
                code_files[fpath] = f.read()
        except (FileNotFoundError, UnicodeDecodeError) as e:
            print(f"Error reading file {fpath}: {e}")
            try:
                with open(fpath, encoding='unicode_escape') as f:
                    code_files[fpath] = f.read()
            except Exception as e2:
                print(f"Error reading file with fallback encoding {fpath}: {e2}")
                continue  # Skip to the next file if reading fails again
        
        try:
            with open(fpath, "rb") as f:
                def get_function_code():
                    tree = parser.parse(f.read())
                    queue = [tree.root_node]
                    while queue:
                        node = queue.pop()
                        if node.type == 'function_definition':
                            function_name = node.child_by_field_name("name").text.decode()
                            function_code = node.text.decode()
                            yield function_name, function_code, node.start_point[0]
                        queue.extend(node.children)
                for func, func_code, func_line in get_function_code():
                    code_funcs[func] = (func_code, func_line)
        except FileNotFoundError:
            print(f"File not found: {fpath}")
            continue  # Skip to the next file if the current one is not found
        
        UNKNOWN = v[1]
        pysteps = []
        for step in v[2:]:
            (
                event, # 0
                timestamp, # 1
                line_no, # 2
                source_line, # 3
                new_vars, # 4
                modified_vars, # 5
                ended_by_exception, # 6
                return_value_repr, # 7
                exception, # 8
            ) = step
            if event == "call":
                m = re.match(r"\s*def\s+([^()\s]+)\s*\(([^)]*)\)", source_line)
                func_name = None
                func_code = None
                func_line = None
                if m:
                    func_name = m.group(1)
                    func_code, func_line = code_funcs.get(func_name, (None, None))

                if len(pysteps) > 0:
                    data.append({
                        "project_name": project_name,
                        "function_name": func_name,
                        "function_code": func_code,
                        "function_line": func_line,
                        "fpath": str(fpath),
                        "steps": pysteps,
                        "code": code_files,
                    })
                pysteps = []
            pysteps.append({
                "event": event,
                "timestamp": timestamp,
                "line_no": line_no,
                "source_line": source_line,
                "new_vars": new_vars,
                "modified_vars": modified_vars,
                "ended_by_exception": ended_by_exception,
                "return_value_repr": return_value_repr,
                "exception": exception,
            })

        # Ensure we don't append empty steps
        if pysteps:
            data.append({
                "fpath": str(fpath),
                "steps": pysteps,
                "code": code_files,
            })

    # Check that all data entries have the 'steps' key
    data = [d for d in data if "steps" in d and "project_name" in d]

    # Create the DataFrame
    pydf = pd.DataFrame(data)
    pydf["language"] = "Python"
    # print(pydf.columns, len(pydf))
    # print(pydf["steps"].str.len().describe())
    # print(pydf["steps"])
    # print(pydf["fpath"].iloc[0])
    # print(pydf["function_name"].notna().mean(), pydf["function_code"].notna().mean())

In [64]:
# Format the Python traces into df.
# Eventually we will have traces from all languages.

df = pydf.copy()
df = df[df["project_name"].notna() & df["function_name"].notna() & df["function_code"].notna() & df["function_line"].notna()]

def format_steps(steps):
    if not steps[0]["source_line"].lstrip().startswith("def"):
        return None

    #function_input = ", ".join([f"{k} = {v[0]}" for k, v in steps[0]["new_vars"].items()])
    function_input = {f"{k}": v[0] for k, v in steps[0]["new_vars"].items()}

    function_output = steps[-1]["return_value_repr"]
    
    
    trace_steps = []
    for i, step in enumerate(steps[:-1]):
        # TODO: Handle exception, jump, multiple assignments within a line, etc.
        # TODO: Handle duplicate lines
        if step["event"] == "exception":
            trace_steps.append({
                "line_no": step["line_no"],
                "values_updated": [{"name": k, "type": v[1], "value": v[0]} for k, v in step["new_vars"].items()],
                "exception": step["exception"]  
            })
            continue
            
        
        #values_updated = steps[i+1]["new_vars"]
        updated_vars = {**steps[i+1].get("new_vars", {}), **steps[i+1].get("modified_vars", {})}
        values_updated = [{"name": k, "type": v[1], "value": v[0]} for k, v in updated_vars.items()]
        trace_steps.append({"line_no": step["line_no"], "values_updated": values_updated})
    return trace_steps, function_input, function_output


# Use `apply` and `zip` to unpack the results from format_steps
df[["trace_steps", "function_input", "function_output"]] = df["steps"].apply(lambda steps: pd.Series(format_steps(steps)))

# Select relevant columns for final DataFrame
df = df[["language", "project_name","function_name", "function_code", "function_line", "trace_steps", "function_input", "function_output"]]

# df = df.drop(columns=["steps"])
# format_steps(df["steps"].iloc[100])
# df = df.rename(columns={"function_name": })
print(len(df), df.columns)
#print(df["trace_steps"].iloc[0][0].keys())

85034 Index(['language', 'project_name', 'function_name', 'function_code',
       'function_line', 'trace_steps', 'function_input', 'function_output'],
      dtype='object')


In [65]:
def format_code(code):
    # Remove blank lines
    code = "".join(line for line in code.splitlines(keepends=True) if line)
    return code


def source_code_formatting(function_code, function_name, function_input):
    lines = function_code.splitlines(keepends=False)
    return "\n".join(lines)

In [66]:
import json
import ast
from mccabe import PathGraphingAstVisitor
from collections import defaultdict

COMMENT_SYMBOL = "#"


# Initialize data structures
seen_sources = set()
jsonl_data = []

for tup in df.itertuples():
    if not isinstance(tup.trace_steps, list):
        continue
    
    # if not filter_complex_entries(tup):
    #     continue
    
    # if tup.function_input is None or any(x in str(tup.function_input).lower() for x in ["none"]):
    #     continue
    
    # if ("tensor" in tup.function_input or "Sequential" in tup.function_input or "array" in tup.function_input or
    #             "ReLU()" in tup.function_input or "Encoder()" in tup.function_input) and len(tup.function_input) > 30:
    #     continue
    
    # Collect value updates on each line
    values_per_line = defaultdict(list)
    repeated_values = defaultdict(list)
    step_index = 1

    variable_name_value_line_mapping = {}


    for variable, value in tup.function_input.items():
        if variable not in variable_name_value_line_mapping:
            variable_name_value_line_mapping[variable] = [] 
        variable_name_value_line_mapping[variable].append((1, value))  


    for step in tup.trace_steps:
        if "exception" in step:
            exception_message = f'EXCEPTION: {step["exception"]}'
            values_per_line[step["line_no"]-tup.function_line].append((step_index, exception_message))
            step_index +=1
        else:
            for value in step["values_updated"]:
                state = f'{value["name"]} = {str(value["value"])}'
                line_position = step["line_no"] - tup.function_line

                if value["name"] in variable_name_value_line_mapping:
                    variable_name_value_line_mapping[value["name"]].append((line_position, value["value"]))
                else:
                    variable_name_value_line_mapping[value["name"]] = [(line_position, value["value"])]

                if state not in repeated_values[line_position]:
                    repeated_values[line_position].append(state)

        
            if len(repeated_values[step["line_no"]-tup.function_line]) > 0:
                state_values = repeated_values[step["line_no"]-tup.function_line]
                values_per_line[step["line_no"]-tup.function_line].append((step_index, state_values))
                step_index += 1
    

    if len(values_per_line) == 0:
        continue
    
    
    source_code_with_input = source_code_formatting(tup.function_code, tup.function_name, tup.function_input)


    lines = tup.function_code.splitlines(keepends=False)
    annotated_lines = []

    function_def_line = lines[0]
    annotated_lines.append(function_def_line)

    for lineno, line in enumerate(lines[1:], start=2):
        states_and_line_values = values_per_line[lineno]
        line = line.rstrip()

        if len(states_and_line_values) > 0 and len(states_and_line_values) == 1:
            for i, (index, values) in enumerate(states_and_line_values):
                for state_index, value in enumerate(values, start=1):
                    line += f" {COMMENT_SYMBOL} [STATE] " + "".join(value) + " [/STATE]"
        annotated_lines.append(line)
        

    source_code_with_input = source_code_with_input.split("\n")
    function_call_with_input = f"{tup.function_name}({', '.join([f'{k}={v}' for k, v in tup.function_input.items()])})"
    annotated_lines.append("\n" + function_call_with_input)
    source_code_with_input.append("\n" + function_call_with_input)

    annotated_function_code = "\n".join(annotated_lines)
    source_code_with_input = "\n".join(source_code_with_input)
    formatted_code = format_code(annotated_function_code)


    if tup.function_name in seen_sources:
        continue

    seen_sources.add(tup.function_name )

    jsonl_data.append({
        "Language": tup.language,
        "Project_Name": tup.project_name,
        "Source Code": source_code_with_input,
        "input" : tup.function_input,
        "output" : tup.function_output,
        "scratchpad_format": formatted_code,
        "variable_values": variable_name_value_line_mapping,
    })


# Write to JSONL file
with open("dataset_all.jsonl", "a", encoding='utf-8') as file:
    for entry in jsonl_data:
        file.write(json.dumps(entry) + "\n")


# Curating again assignment

In [1]:
import re
import tiktoken

def contains_complex_type(value):
    """Check if a string contains complex types."""
    complex_patterns = [
        r"<class '.*'>",               # e.g., "<class 'peewee.SqliteDatabase'>"
        r"<.*object at 0x[\da-f]+>",    # e.g., "<peewee.PrimaryKeyField object at 0x7f9ad1c4d550>"
        r"at 0x[\da-f]+",               # any memory address
        r"^\s*<",                       # any string starting with "<"
    ]
    
    if isinstance(value, str):
        for pattern in complex_patterns:
            if re.search(pattern, value):
                return True
    return False



def token_filtering(source_code):
    tokenizer = tiktoken.get_encoding("cl100k_base") 
    MAX_TOKENS = 2048  


    token_count = len(tokenizer.encode(source_code))

    if token_count <= MAX_TOKENS:  
        return True
    else:
        return False


In [68]:
import random
import json
import tree_sitter
from tree_sitter_languages import get_parser
import re

parser = get_parser("python")


"Having assignments with only <STATE> comments"
def filtering_assignments(assignments, annotated_lines):
    filtered_assingments = []
    for assignment in assignments:
        line_no = assignment.start_point[0]

        if "# [STATE]" in annotated_lines[line_no]:
            #comment = "# "+annotated_lines[line_no].split("#")[1]
            filtered_assingments.append(assignment)
    return filtered_assingments


def get_last_value(var_name, variable_values, current_line):
    """Fetch the last recorded value of a variable before a given line."""
    if var_name in variable_values:
        values = variable_values[var_name]
        for line, value in reversed(values):
            if line < current_line:
                return value
    return None

def find_assignments(node):
    """Recursively find all assignment nodes in the AST."""
    assignments = []
    for child in node.children:
        if child.type == "assignment" and node.type == "expression_statement":
            assignments.append(child)
        assignments.extend(find_assignments(child))
    
    return assignments

def is_immediate_value(rhs_text):
    """Check if the RHS is an immediate value like int, float, string, list, tuple, dict."""
    try:
        eval_rhs = eval(rhs_text)  # Evaluate safely (controlled usage)
        return isinstance(eval_rhs, (int, float, str, list, tuple, dict, bool))
    except:
        return False  # If eval fails, it's not an immediate value

def process_assignment(node, variable_values):
    """Process an assignment expression and generate comments/statements."""
    lhs = node.child_by_field_name("left")
    rhs = node.child_by_field_name("right")
    if lhs is None or rhs is None:
        return None, None, None
    
    rhs_text = rhs.text.decode()
    current_line = node.start_point[0] + 1  # Tree-sitter lines start from 0
    
    if "(" in rhs_text or is_immediate_value(rhs_text):  
        return None, None, None
    
    identifiers = [child.text.decode() for child in rhs.children if child.type == "identifier"]
    values = {var: get_last_value(var, variable_values, current_line) for var in identifiers}


    if len(identifiers) == 0:
        return None, None, None
    
    if all(v is not None for v in values.values()):
        comment = f"# {' '.join(f'{k} = {v}' for k, v in values.items())}"
        statement = f"The value of '{', '.join(values.keys())}' is '{', '.join(map(str, values.values()))}' before the execution of statement \"{lhs.text.decode()} = {rhs_text}\". Can you predict the value of \"{lhs.text.decode()}\" after the execution of the statement?"
        return comment, statement, values
    
    return None, None, None

def annotate_code(example):
    """Parse and annotate the code with extracted variable values."""
    code = example["Source Code"]
    source_code = example["scratchpad_format"]
    variable_values = example["variable_values"]
    tree = parser.parse(source_code.encode())

    code_lines = code.split("\n")
    annotated_lines = source_code.split("\n")
    statements = []

    assignments = find_assignments(tree.root_node)

    filtered_assignments = filtering_assignments(assignments, annotated_lines)


    if not filtered_assignments:
        return None, None, None, None  # No assignments found
    
    # Try multiple assignments until we find one that is valid
    valid_assignment = None
    for assignment in assignments:
        comment, statement, values = process_assignment(assignment, variable_values)
        if comment and statement:
            valid_assignment = assignment
            break 
    if valid_assignment:
        line_number = valid_assignment.start_point[0]


        match_item = re.search(r'\# \[STATE\](.*?)\[/STATE\]', annotated_lines[line_number], re.DOTALL)

        ground_truth = None
        if match_item:
            ground_truth = match_item.group(1).strip()
        else:
            ground_truth = None   

        return "\n".join(code_lines), code_lines[line_number], ground_truth, values
    
    return None, None, None, None

# Example usage
with open("dataset_all.jsonl", "r") as f:
    dataset = [json.loads(line) for line in f]

filtered_results = []

for example in dataset:
    if any(contains_complex_type(val) for val in example["input"].values()):
        continue


    annotated_code, statements, ground_truth, values = annotate_code(example)
    if annotated_code and statements and ground_truth: 
        if contains_complex_type(ground_truth) or not token_filtering(annotated_code):
            continue
        if "=" in ground_truth:
            filtered_results.append(
                {"Programming Language":example["Language"],
                "Statement Type": "Assignment",
                "Source Code": annotated_code, 
                "Selected Statement": statements.strip(),
                "Function Input": example["input"],
                "Variable Values Before Statement": values, 
                "Value After Statement Execution": ground_truth.split("=")[1].strip(),
                "Variable States During Runtime": example["variable_values"],
                "Program Information": f"Project Name: {example['Project_Name']}"
                })


with open("Unified_dataset_assignment.jsonl", "w") as f:
    for result in filtered_results:
        f.write(json.dumps(result) + "\n")


# Constant Assignment

In [2]:
import random
import json
import tree_sitter
from tree_sitter_languages import get_parser
import re

parser = get_parser("python")


"Having assignments with only <STATE> comments"
def filtering_assignments(assignments, annotated_lines):
    filtered_assingments = []
    for assignment in assignments:
        line_no = assignment.start_point[0]

        if "# [STATE]" in annotated_lines[line_no]:
            #comment = "# "+annotated_lines[line_no].split("#")[1]
            filtered_assingments.append(assignment)
    return filtered_assingments


def get_last_value(var_name, variable_values, current_line):
    """Fetch the last recorded value of a variable before a given line."""
    if var_name in variable_values:
        values = variable_values[var_name]
        for line, value in reversed(values):
            if line < current_line:
                return value
    return None

def find_assignments(node):
    """Recursively find all assignment nodes in the AST."""
    assignments = []
    for child in node.children:
        if child.type == "assignment" and node.type == "expression_statement":
            assignments.append(child)
        assignments.extend(find_assignments(child))
    
    return assignments


def is_constant_value(rhs_text):
    """Check if the RHS is a constant value like int, float, string, bool."""
    try:
        eval_rhs = eval(rhs_text)  # Evaluate safely (controlled usage)
        return isinstance(eval_rhs, (int, float, str, bool))
    except:
        return False  # If eval fails, it's not a constant value

def process_assignment(node, variable_values):
    """Process an assignment expression and generate comments/statements."""
    lhs = node.child_by_field_name("left")
    rhs = node.child_by_field_name("right")
    if lhs is None or rhs is None:
        return None, None, None
    
    rhs_text = rhs.text.decode()
    current_line = node.start_point[0] + 1  # Tree-sitter lines start from 0
    
    # Only process if RHS is a constant value
    if not is_constant_value(rhs_text):
        return None, None, None
    
    # For constant assignments, we don't need to track variable values
    comment = f"# Constant assignment: {lhs.text.decode()} = {rhs_text}"
    statement = f"The value of '{lhs.text.decode()}' is assigned the constant value '{rhs_text}'."
    values = {"Constant": rhs_text}
    
    return comment, statement, values

def annotate_code(example):
    """Parse and annotate the code with extracted variable values."""
    code = example["Source Code"]
    source_code = example["scratchpad_format"]
    variable_values = example["variable_values"]
    tree = parser.parse(source_code.encode())

    code_lines = code.split("\n")
    annotated_lines = source_code.split("\n")
    statements = []

    assignments = find_assignments(tree.root_node)

    filtered_assignments = filtering_assignments(assignments, annotated_lines)

    if not filtered_assignments:
        return None, None, None, None  # No assignments found
    
    # Try multiple assignments until we find one that is valid
    valid_assignment = None
    for assignment in assignments:
        comment, statement, values = process_assignment(assignment, variable_values)
        if comment and statement:
            valid_assignment = assignment
            break 
    if valid_assignment:
        line_number = valid_assignment.start_point[0]

        match_item = re.search(r'\# \[STATE\](.*?)\[/STATE\]', annotated_lines[line_number], re.DOTALL)

        ground_truth = None
        if match_item:
            ground_truth = match_item.group(1).strip()
        else:
            ground_truth = None   

        return "\n".join(code_lines), code_lines[line_number], ground_truth, values
    
    return None, None, None, None

# Example usage
with open("dataset_all.jsonl", "r") as f:
    dataset = [json.loads(line) for line in f]

filtered_results = []

for example in dataset:
    if any(contains_complex_type(val) for val in example["input"].values()):
        continue

    annotated_code, statements, ground_truth, values = annotate_code(example)
    if annotated_code and statements and ground_truth: 
        if contains_complex_type(ground_truth) or not token_filtering(annotated_code):
            continue
        if "=" in ground_truth:
            filtered_results.append(
                {"Programming Language": example["Language"],
                 "Statement Type": "Constant Assignment",
                 "Source Code": annotated_code, 
                 "Selected Statement": statements.strip(),
                 "Function Input": example["input"],
                 "Variable Values Before Statement": values, 
                 "Value After Statement Execution": values["Constant"],  # Constant value as string
                 "Variable States During Runtime": example["variable_values"],
                 "Program Information": f"Project Name: {example['Project_Name']}"
                })

with open("Unified_dataset_constant_assignment.jsonl", "w") as f:
    for result in filtered_results:
        f.write(json.dumps(result) + "\n")

# Arithmatic Assignment

In [5]:
import random
import json
import tree_sitter
from tree_sitter_languages import get_parser
import re

parser = get_parser("python")


"Having assignments with only <STATE> comments"
def filtering_assignments(assignments, annotated_lines):
    filtered_assingments = []
    for assignment in assignments:
        line_no = assignment.start_point[0]

        if "# [STATE]" in annotated_lines[line_no]:
            #comment = "# "+annotated_lines[line_no].split("#")[1]
            filtered_assingments.append(assignment)
    return filtered_assingments


def get_last_value(var_name, variable_values, current_line):
    """Fetch the last recorded value of a variable before a given line."""
    if var_name in variable_values:
        values = variable_values[var_name]
        for line, value in reversed(values):
            if line < current_line:
                return value
    return None

def find_assignments(node):
    """Recursively find all assignment nodes in the AST."""
    assignments = []
    for child in node.children:
        if child.type == "assignment" and node.type == "expression_statement":
            assignments.append(child)
        assignments.extend(find_assignments(child))
    
    return assignments

def is_arithmetic_expression(rhs_node):
    """Check if the RHS is an arithmetic expression."""
    arithmetic_operators = {"+", "-", "*", "/", "//", "%", "**"}  # Supported arithmetic operators
    if rhs_node.type == "binary_operator":
        operator = rhs_node.child_by_field_name("operator").text.decode()
        return operator in arithmetic_operators
    return False

def process_arithmetic_assignment(node, variable_values):
    """Process an arithmetic assignment expression and generate comments/statements."""
    lhs = node.child_by_field_name("left")
    rhs = node.child_by_field_name("right")
    if lhs is None or rhs is None:
        return None, None, None
    
    rhs_text = rhs.text.decode()
    current_line = node.start_point[0] + 1  # Tree-sitter lines start from 0
    
    # Only process if RHS is an arithmetic expression
    if not is_arithmetic_expression(rhs):
        return None, None, None
    
    # Extract variables used in the arithmetic expression
    identifiers = [child.text.decode() for child in rhs.children if child.type == "identifier"]
    values = {var: get_last_value(var, variable_values, current_line) for var in identifiers}

    # Skip if any variable value is missing
    if any(v is None for v in values.values()):
        return None, None, None
    
    # Generate comment and statement
    comment = f"# Arithmetic assignment: {lhs.text.decode()} = {rhs_text}"
    statement = f"The value of '{lhs.text.decode()}' is assigned the result of the arithmetic expression '{rhs_text}'."
    
    return comment, statement, values

def annotate_code(example):
    """Parse and annotate the code with extracted variable values."""
    code = example["Source Code"]
    source_code = example["scratchpad_format"]
    variable_values = example["variable_values"]
    tree = parser.parse(source_code.encode())

    code_lines = code.split("\n")
    annotated_lines = source_code.split("\n")
    statements = []

    assignments = find_assignments(tree.root_node)

    filtered_assignments = filtering_assignments(assignments, annotated_lines)

    if not filtered_assignments:
        return None, None, None, None  # No assignments found
    
    # Try multiple assignments until we find one that is valid
    valid_assignment = None
    for assignment in assignments:
        comment, statement, values = process_arithmetic_assignment(assignment, variable_values)
        if comment and statement:
            valid_assignment = assignment
            break 
    if valid_assignment:
        line_number = valid_assignment.start_point[0]

        match_item = re.search(r'\# \[STATE\](.*?)\[/STATE\]', annotated_lines[line_number], re.DOTALL)

        ground_truth = None
        if match_item:
            ground_truth = match_item.group(1).strip()
        else:
            ground_truth = None   

        return "\n".join(code_lines), code_lines[line_number], ground_truth, values
    
    return None, None, None, None

# Example usage
with open("dataset_all.jsonl", "r") as f:
    dataset = [json.loads(line) for line in f]

filtered_results = []

for example in dataset:
    if any(contains_complex_type(val) for val in example["input"].values()):
        continue

    annotated_code, statements, ground_truth, values = annotate_code(example)
    if values is None:
        continue
    
    if annotated_code and statements and ground_truth: 
        if contains_complex_type(ground_truth) or not token_filtering(annotated_code):
            continue
        if "=" in ground_truth:
            filtered_results.append(
                {"Programming Language": example["Language"],
                 "Statement Type": "Arithmetic Assignment",
                 "Source Code": annotated_code, 
                 "Selected Statement": statements.strip(),
                 "Function Input": example["input"],
                 "Variable Values Before Statement": values, 
                 "Value After Statement Execution": ground_truth.split("=")[1].strip(),
                 "Variable States During Runtime": example["variable_values"],
                 "Program Information": f"Project Name: {example['Project_Name']}"
                })

with open("Unified_dataset_arithmetic_assignment.jsonl", "w") as f:
    for result in filtered_results:
        f.write(json.dumps(result) + "\n")


# Branch

In [69]:
import random
import json
import tree_sitter
from tree_sitter_languages import get_parser
import re

parser = get_parser("python")

def filtering_branches(branches, annotated_lines):
    yes_branches = []
    no_branches = []
    for branch in branches:
        line_no = branch.start_point[0]

        if "# [STATE]" in annotated_lines[line_no + 1]:
            yes_branches.append(branch)
        else:
            no_branches.append(branch)
    return yes_branches, no_branches


def get_last_value(var_name, variable_values, current_line):
    """Fetch the last recorded value of a variable before a given line."""
    if var_name in variable_values:
        values = variable_values[var_name]
        for line, value in reversed(values):
            if line < current_line:
                return value
    return None


def find_branches(node):
    """Recursively find all if-statements in the AST."""
    branches = []

    if node.type == "if_statement":
        branches.append(node)

    for child in node.children:
        branches.extend(find_branches(child))

    return branches


def process_branches(node, variable_values):
    """Process branch expression and generate comments/statements."""

    current_line = node.start_point[0] + 1  # Tree-sitter lines start from 0

    if len(node.children) > 1:
        condition_node = node.children[1]
        condition_text = condition_node.text.decode()
        current_line = condition_node.start_point[0]
    else:
        condition_text = node.text.decode()
        current_line = node.start_point[0]

    identifiers = [
        child.text.decode()
        for child in condition_node.children
        if child.type == "identifier"
    ]
    values = {var: get_last_value(var, variable_values, current_line) for var in identifiers}

    if not identifiers:
        return None, None, None, None

    if all(v is not None for v in values.values()):
        comment = f"# {' '.join(f'{k} = {v}' for k, v in values.items())}"
        statement = (
            f"The value of '{', '.join(values.keys())}' is '{', '.join(map(str, values.values()))}' "
            f"before evaluating 'if {condition_text}'. Can you predict whether this branch will be executed?"
        )
        return comment, statement, current_line, values

    return None, None, None, None


def annotate_code(example, yes_count, no_count, max_per_category):
    """Parse and annotate the code with extracted variable values while ensuring balance."""
    code = example["Source Code"]
    source_code = example["scratchpad_format"]
    variable_values = example["variable_values"]
    tree = parser.parse(source_code.encode())

    code_lines = code.split("\n")
    annotated_lines = source_code.split("\n")

    branches = find_branches(tree.root_node)
    yes_branches, no_branches = filtering_branches(branches, annotated_lines)

    chosen_branch_type = None
    chosen_branch_list = None
    ground_truth = None

    if yes_branches and (yes_count < max_per_category or not no_branches):
        chosen_branch_type = "Yes"
        chosen_branch_list = yes_branches
    elif no_branches and (no_count < max_per_category or not yes_branches):
        chosen_branch_type = "No"
        chosen_branch_list = no_branches

    if not chosen_branch_list:
        return None, None, None, yes_count, no_count, None  


    branch = random.choice(chosen_branch_list)
    comment, statement, line_number, values = process_branches(branch, variable_values)

    if comment and statement and line_number:

        if chosen_branch_type == "Yes":
            yes_count += 1
        else:
            no_count += 1

        return "\n".join(code_lines), code_lines[line_number] , chosen_branch_type, yes_count, no_count, values

    return None, None, None, yes_count, no_count, None



with open("dataset_all.jsonl", "r") as f:
    dataset = [json.loads(line) for line in f]


filtered_results = []
yes_count = 0
no_count = 0
max_per_category = len(dataset) // 2  # Let's make the dataset balanced

for example in dataset:
    if any(contains_complex_type(val) for val in example["input"].values()):
        continue
    annotated_code, statements, ground_truth, yes_count, no_count, values = annotate_code(
        example, yes_count, no_count, max_per_category
    )
    if annotated_code and statements and ground_truth:
        if contains_complex_type(ground_truth) or not token_filtering(annotated_code):
            continue
        filtered_results.append(
            {"Programming Language":example["Language"],
            "Statement Type": "Branch",
            "Source Code": annotated_code, 
            "Selected Statement": statements.strip(),
            "Function Input": example["input"],
            "Variable Values Before Statement": values, 
            "Value After Statement Execution": ground_truth,
            "Variable States During Runtime": example["variable_values"],
            "Program Information": f"Project Name: {example['Project_Name']}"
            })


with open("Unified_dataset_branch.jsonl", "w") as f:
    for result in filtered_results:
        f.write(json.dumps(result) + "\n")



In [70]:
import random
import json
import tree_sitter
from tree_sitter_languages import get_parser
import re

parser = get_parser("python")

def function_filtering(function_node):
    function_name = function_node.child_by_field_name("function")
    arguments_node = function_node.child_by_field_name("arguments") 

    function_name_text = function_name.text.decode() if function_name else "Unknown"


    identifiers = [
    child.text.decode()
    for child in arguments_node.children
    if child.type == "identifier"]

    if len(identifiers) == 0:
        return False

    return True

def filtering_function_calls(function_calls, annotated_lines):
    filtered_function_calls = []
    for function in function_calls:
        line_no = function.start_point[0]
        if "# [STATE]" in annotated_lines[line_no]:
            does_match_requirements = function_filtering(function)
            if does_match_requirements:
                filtered_function_calls.append(function)
    return filtered_function_calls


def get_last_value(var_name, variable_values, current_line):
    """Fetch the last recorded value of a variable before a given line."""
    if var_name in variable_values:
        values = variable_values[var_name]
        for line, value in reversed(values):
            if line < current_line:
                return value
    return None


def find_function_calls(node):
    """Recursively find all function call expressions in the AST, ignoring function definitions."""
    calls = []

    if node.type == "call": 
        calls.append(node)

    # if node.type == "function_definition":
    #     return calls

    for child in node.children:
        calls.extend(find_function_calls(child))

    return calls


def process_calls(function_node, variable_values):
    """Process API calls expression and generate comments/statements."""

    current_line = function_node.start_point[0] + 1 
    arguments_node = function_node.child_by_field_name("arguments") 

    identifiers = [
    child.text.decode()
    for child in arguments_node.children
    if child.type == "identifier"]
    
    values = {var: get_last_value(var, variable_values, current_line) for var in identifiers}


    if all(v is not None for v in values.values()):
        comment = f"# {' '.join(f'{k} = {v}' for k, v in values.items())}"
        statement = f"The value of the parameter(s) '{', '.join(values.keys())}' is '{', '.join(map(str, values.values()))}' before the execution of function \"{function_node.text.decode()}\". Can you predict the output of the function \"{function_node.text.decode()}\" with the given parameter(s) value(s)?"
        return comment, statement, values
    
    return None, None, None


def annotate_code(example):
    """Parse and curate the dataset"""
    code = example["Source Code"]
    source_code = example["scratchpad_format"]
    variable_values = example["variable_values"]
    tree = parser.parse(code.encode())

    code_lines = code.split("\n")
    annotated_lines = source_code.split("\n")


    function_calls = find_function_calls(tree.root_node)
    filtered_function_calls = filtering_function_calls(function_calls, annotated_lines)

    if len(filtered_function_calls) == 0:
        return None, None, None, None

    chosen_function = random.choice(filtered_function_calls)
    comment, statement, values = process_calls(chosen_function, variable_values)

    if not comment or not statement:
        return None, None, None, None
    
    line_number = chosen_function.start_point[0]


    match_item = re.search(r'\# \[STATE\](.*?)\[/STATE\]', annotated_lines[line_number], re.DOTALL)

    ground_truth = None
    if match_item:
        ground_truth = match_item.group(1).strip()
    else:
        ground_truth = None   

    return "\n".join(code_lines), code_lines[line_number], ground_truth, values


with open("dataset_all.jsonl", "r") as f:
    dataset = [json.loads(line) for line in f]



filtered_results = []
for example in dataset:
    if any(contains_complex_type(val) for val in example["input"].values()):
        continue
    code, statements, ground_truth, values = annotate_code(example)
    if code and statements and ground_truth and values:
        if contains_complex_type(ground_truth) or not token_filtering(code):
            continue
        if "=" in ground_truth:
            print(code)
            print(statements.strip())
            print(values)
            print(ground_truth)

            filtered_results.append(
                {"Programming Language":example["Language"],
                "Statement Type": "API",
                "Source Code": code, 
                "Selected Statement": statements.strip(),
                "Function Input": example["input"],
                "Variable Values Before Statement": values, 
                "Value After Statement Execution": ground_truth.split("=")[1].strip(),
                "Variable States During Runtime": example["variable_values"],
                "Program Information": f"Project Name: {example['Project_Name']}"
                })

with open("Unified_dataset_api.jsonl", "w") as f:
    for result in filtered_results:
        f.write(json.dumps(result) + "\n")



def secrets_dir(env=os.getenv('D2_ENVIRONMENT', None),
                basedir=os.getenv('D2_SECRETS_BASEDIR', None)):
    if env is not None:
        env_str = str(env)
    else:
        cwd = os.getcwd()
        default_file = os.path.join(cwd, '.python_secrets_environment')
        if os.path.exists(default_file):
            with open(default_file, 'r') as f:
                env_str = f.read().strip()
        else:
            env_str = os.path.basename(cwd)
    if basedir is None:
        basedir = os.path.join(
                HOME,
                'secrets' if sys.platform.startswith('win') else '.secrets')
    return os.path.join(basedir, env_str)

secrets_dir(env=None, basedir=None)
default_file = os.path.join(cwd, '.python_secrets_environment')
{'cwd': "'/local/rcs/XXX/code/pytrace-collector/logs/pypibugs/tried/davedittrich+python_secrets/davedittrich+python_secrets'"}
default_file = '/local/rcs/XXX/code/pytrace-collector/logs/pypibugs/tried/davedittrich+python_secrets/davedi

In [6]:
import json

# Input JSONL file paths
input_files = ["Unified_dataset_assignment.jsonl", "Unified_dataset_branch.jsonl", "Unified_dataset_api.jsonl", "Unified_dataset_arithmetic_assignment.jsonl", "Unified_dataset_constant_assignment.jsonl"]
output_file = "statement_prediction_dataset_1.jsonl"

merged_data = []

# Read each file and append its content
for file in input_files:
    with open(file, "r") as f:
        for line in f:
            data = json.loads(line)  # Parse JSONL entry
            merged_data.append(data)

# Add idx field starting from 1
for idx, entry in enumerate(merged_data, start=1):
    entry["idx"] = idx

# Write the merged data into a new JSONL file
with open(output_file, "w") as f:
    for entry in merged_data:
        f.write(json.dumps(entry) + "\n")

print(f"Merged {len(merged_data)} entries into {output_file}")


Merged 545 entries into statement_prediction_dataset_1.jsonl
